In [1]:
import numpy as np
import xarray as xr

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

Files: ERA5 pressure level data from `1979-01-01T00 - 1979-01-01T12`

* `/glade/derecho/scratch/ksha/CREDIT_data/ERA5_pressure_lev/upper_air_test.zarr`
* `/glade/derecho/scratch/ksha/CREDIT_data/ERA5_pressure_lev/surf_test.zarr`

In [3]:
ds_upper = xr.open_zarr('/glade/derecho/scratch/ksha/CREDIT_data/ERA5_pressure_lev/upper_air_test.zarr')
ds_surf = xr.open_zarr('/glade/derecho/scratch/ksha/CREDIT_data/ERA5_pressure_lev/surf_test.zarr')

In [4]:
GRAVITY = 9.80665
LH_WATER = 2.26e6  # J/kg
CP_DRY = 1005 # J/kg K
CP_VAPOR = 1846 # J/kg K
R = 6371000  # m

In [5]:
x = ds_surf['longitude']
y = ds_surf['latitude']
lon, lat = np.meshgrid(x, y)
level_p = 100*np.array(ds_upper['level'])

In [6]:
level_p # Pa or kg/m/s2

array([   100,    200,    300,    500,    700,   1000,   2000,   3000,
         5000,   7000,  10000,  12500,  15000,  17500,  20000,  22500,
        25000,  30000,  35000,  40000,  45000,  50000,  55000,  60000,
        65000,  70000,  75000,  77500,  80000,  82500,  85000,  87500,
        90000,  92500,  95000,  97500, 100000])

In [7]:
q = np.array(ds_upper['specific_humidity']) # kg/kg
T = np.array(ds_upper['temperature']) # K
u = np.array(ds_upper['u_component_of_wind']) # m/s
v = np.array(ds_upper['v_component_of_wind'])
GPH_surf = np.array(ds_surf['geopotential_at_surface']) # J/m2
TOA_net = np.array(ds_surf['top_net_solar_radiation']) # J/m2
OLR = np.array(ds_surf['top_net_thermal_radiation']) # J/m2
R_short = np.array(ds_surf['surface_net_solar_radiation']) # J/m2
R_long = np.array(ds_surf['surface_net_thermal_radiation']) # J/m2
LH = np.array(ds_surf['surface_latent_heat_flux']) # J/m2
SH = np.array(ds_surf['surface_sensible_heat_flux']) # J/m2

In [8]:
def weighted_sum(data, weights, axis, keepdims=False):
    '''
    Compute the weighted sum of a given quantity

    Args:
        data: the quantity to be sum-ed
        weights: weights that can be broadcasted to the shape of data
        axis: dims to compute the sum
        keepdims: keepdims

    Returns:
        weighted sum
    '''
    expanded_weights = np.broadcast_to(weights, data.shape)
    return np.sum(data * expanded_weights, axis=axis, keepdims=keepdims)

def pressure_integral(q, level_p, output_shape):
    '''
    Compute the pressure level integral of a given quantity using np.trapz

    Args:
        q: the quantity with dims of (level, lat, lon) or (time, level, lat, lon)
        level_p: the pressure level of q as [Pa] and with dims of (level,)
        output_shape: either (lat, lon) or (time, lat, lon)

    Returns:
        Pressure level integrals of q
    '''
    # (level, lat, lon) --> (lat, lon)
    if len(output_shape) == 2:
        Q = np.empty(output_shape)
        for ix in range(output_shape[0]):
            for iy in range(output_shape[1]):
                Q[ix, iy] = np.trapz(q[:, ix, iy], level_p)
                
    # (time, level, lat, lon) --> (time, lat, lon)
    elif len(output_shape) == 3:
        Q = np.empty(output_shape)
        for i_time in range(output_shape[0]):
            for ix in range(output_shape[1]):
                for iy in range(output_shape[2]):
                    Q[i_time, ix, iy] = np.trapz(q[i_time, :, ix, iy], level_p)
                    
    else:
        print('wrong output_shape')
        raise
        
    return Q
    
def dx_dy(lat, lon):
    '''
    Compute the grid spacing from 2D lat/lon grids using central difference
    for center grids and forward/backward difference for edge grids

    Args:
        lat, lon: 2D arrays of latitude and longitude.

    Return:
        dy, dx: 2D arrays of grid spacings
    '''
    
    # Convert latitude and longitude from degrees to radians
    lat_rad = np.radians(lat)
    lon_rad = np.radians(lon)
    
    # Compute the grid spacing in the latitude direction (dy)
    dy = np.zeros_like(lat)
    dy[1:-1, :] = R * (lat_rad[2:, :] - lat_rad[:-2, :]) / 2.0
    dy[0, :] = R * (lat_rad[1, :] - lat_rad[0, :])
    dy[-1, :] = R * (lat_rad[-1, :] - lat_rad[-2, :])
    
    # Compute the grid spacing in the longitude direction (dx)
    dx = np.zeros_like(lon)
    dx[:, 1:-1] = R * np.cos(lat_rad[:, 1:-1]) * (lon_rad[:, 2:] - lon_rad[:, :-2]) / 2.0
    dx[:, 0] = R * np.cos(lat_rad[:, 0]) * (lon_rad[:, 1] - lon_rad[:, 0])
    dx[:, -1] = R * np.cos(lat_rad[:, -1]) * (lon_rad[:, -1] - lon_rad[:, -2])

    return dy, dx

def compute_grid_area(lat, lon):
    '''
    Compute grid cell areas from 2D lat/lon grids

    Args:
        lat, lon: latitude and longitude with dims of (lat, lon)

    Return:
        grid cell area, dims are (lat, lon)
    '''
    dy, dx = dx_dy(lat, lon)
    area = dy*dx

    return area

area = compute_grid_area(lat, lon)

# w_lat = np.cos(np.deg2rad(lat))
w_lat = area #/ np.sum(area)

## Conservation of energy

Reference:
* [Trenberth and Solomon 1994](https://link.springer.com/content/pdf/10.1007/BF00210625.pdf)
* [Atmospheric conservation properties
in ERA-Interim](https://www.ecmwf.int/sites/default/files/elibrary/2011/8175-atmospheric-conservation-properties-era-interim.pdf)
* [ERA5 variable table](https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-single-levels?tab=overview)


Equation on a single air-column:

\begin{equation}
\frac{\partial \mathrm{E}}{\partial t} + \mathbf{\nabla}\cdot\frac{1}{g}\int_{p_0}^{p_1}{\mathbf{v}\left(h+k\right)}dp = R_T - F_S
\end{equation}

$E$ is the total energy of an air column

\begin{equation}
E = \frac{1}{g}\int_{p_0}^{p_1}{\left(Lq+C_pT+\Phi_s+k\right)}dp
\end{equation}

Where $L$ is latent heat of condensation of water, $C_p$ is  the specific heat capacity of air at constant pressure, $\Phi_s$ is geopotential at surface, $k=0.5*\left(\mathbf{v} \cdot \mathbf{v}\right)$ is kinetic energy.

$C_p$ is a function of humidity:

\begin{equation}
C_p = (1-q) C_{pd} + q C_{pv}
\end{equation}

$R_T$ and $F_S$ are energy fluxes on the top of the atmosphere and the surface of earth:

\begin{equation}
R_T = TOA_{\mathrm{net}} + OLR
\end{equation}

\begin{equation}
F_s = R_{\mathrm{short}} + R_{\mathrm{long}} + H_{\mathrm{sensible}} + H_{\mathrm{latent}}
\end{equation}

For global sum, the divergence term is 0, so the time tendency of total energy is balanced by energy fluxes from the top of the atmosphere and the surface of earth:

\begin{equation}
\overline{\left(\frac{\partial E}{\partial t}\right)} =  \overline{R_T} - \overline{F_s}
\end{equation}

In [9]:
t0 = 8
t1 = 9
N_seconds = 3600

In [10]:
C_p = (1-q)*CP_DRY + q*CP_VAPOR
C_p = C_p[:t1+1, ...]

ken = 0.5*(u**2 + v**2)
ken = ken[:t1+1, ...]

T_correct = np.copy(T)
T_correct = T_correct[:t1+1, ...]

GPH_surf = GPH_surf[:t1+1, ...]

q = q[:t1+1, ...]

In [12]:
TE_temp_term.shape

(2,)

In [11]:
correction_cycle_num = 4

for i in range(correction_cycle_num):

    TE_temp_term = weighted_sum(
        pressure_integral(
            C_p * T_correct, level_p, (2,)+lon.shape) / GRAVITY, 
        w_lat, axis=(1, 2)) 
    
    TE_vapor_ken = weighted_sum(
        pressure_integral(
            LH_WATER*q + GPH_surf[:, None, ...] + ken, level_p, (2,)+lon.shape) / GRAVITY, 
        w_lat, axis=(1, 2)) 
    
    TE_sum_t1 = TE_temp_term[t1, ...] + TE_vapor_ken[t1, ...] 
    TE_sum_t0 = TE_temp_term[t0, ...] + TE_vapor_ken[t0, ...]
    dTE_sum = (TE_sum_t1 - TE_sum_t0) / N_seconds
    
    R_T = (TOA_net + OLR) / N_seconds
    R_T = R_T[t1, ...]
    
    F_S = (R_short + R_long + LH + SH) / N_seconds
    F_S = F_S[t1, ...]
    
    RF_sum = weighted_sum(R_T+F_S, w_lat, axis=(0, 1), keepdims=False)
    energy_residual = dTE_sum + RF_sum
    print('Residual to conserve energy budge [J/s]: {}'.format(energy_residual))
    
    T_correct_ratio = (-RF_sum * N_seconds + TE_sum_t0 - TE_vapor_ken[t1, ...]) / TE_temp_term[t1, ...]
    T_correct[t1, ...] = T_correct[t1, ...]  * T_correct_ratio
    print('Correction ratio: {}'.format(T_correct_ratio))
    
TE_temp_term = weighted_sum(
    pressure_integral(
        C_p * T_correct, level_p, (2,)+lon.shape) / GRAVITY, 
    w_lat, axis=(1, 2)) 

TE_vapor_ken = weighted_sum(
    pressure_integral(
        LH_WATER*q + GPH_surf[:, None, ...] + ken, level_p, (2,)+lon.shape) / GRAVITY, 
    w_lat, axis=(1, 2)) 

TE_sum_t1 = TE_temp_term[t1, ...] + TE_vapor_ken[t1, ...] 
TE_sum_t0 = TE_temp_term[t0, ...] + TE_vapor_ken[t0, ...]
dTE_sum = (TE_sum_t1 - TE_sum_t0) / N_seconds

R_T = (TOA_net + OLR) / N_seconds
R_T = R_T[t1, ...]

F_S = (R_short + R_long + LH + SH) / N_seconds
F_S = F_S[t1, ...]

RF_sum = weighted_sum(R_T+F_S, w_lat, axis=(0, 1), keepdims=False)
energy_residual = dTE_sum + RF_sum
print('Residual to conserve energy budge [J/s]: {}'.format(energy_residual))

IndexError: index 9 is out of bounds for axis 0 with size 2

In [ ]:
# # an example of std(q) that varies on pressure levels
# # top of atmos --> surface
# T_std = np.array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
#                   4, 4, 4, 4, 4, 2, 2])

# T_std_norm = T_std / np.mean(T_std) # geometric_mean(q_std) # q_w

In [ ]:
C_p = (1-q)*CP_DRY + q*CP_VAPOR
C_p = C_p[:t1+1, ...]

ken = 0.5*(u**2 + v**2)
ken = ken[:t1+1, ...]

T_correct = np.copy(T)
T_correct = T_correct[:t1+1, ...]

GPH_surf = GPH_surf[:t1+1, ...]

q = q[:t1+1, ...]

In [ ]:
correction_cycle_num = 6

for i in range(correction_cycle_num):

    E_level = C_p * T_correct + LH_WATER*q + GPH_surf[:, None, ...] + ken
    TE = pressure_integral(E_level, level_p, (2,)+lon.shape) / GRAVITY
    
    dTE_dt = (TE[1, ...] - TE[0, ...]) / N_seconds
    dTE_sum = weighted_sum(dTE_dt, w_lat, axis=(0, 1), keepdims=False)
    
    R_T = (TOA_net + OLR) / N_seconds
    R_T = R_T[t1, ...]
    
    F_S = (R_short + R_long + LH + SH) / N_seconds
    F_S = F_S[t1, ...]
    
    RF_sum = weighted_sum(R_T+F_S, w_lat, axis=(0, 1), keepdims=False)
    energy_residual = dTE_sum + RF_sum
    print('Residual to conserve energy budge [J/s]: {}'.format(energy_residual))
    
    # T_correct_ratio = 1 + T_std_norm * (-RF_sum/dTE_sum - 1)
    T_correct_ratio = -RF_sum/dTE_sum
    T_correct = T_correct * T_correct_ratio
    print('Correction ratio: {}'.format(T_correct_ratio))

E_level = C_p * T_correct + LH_WATER*q + GPH_surf[:, None, ...] + ken
TE = pressure_integral(E_level, level_p, (2,)+lon.shape) / GRAVITY

dTE_dt = (TE[1, ...] - TE[0, ...]) / N_seconds
dTE_sum = weighted_sum(dTE_dt, w_lat, axis=(0, 1), keepdims=False)

R_T = (TOA_net + OLR) / N_seconds
R_T = R_T[t1, ...]

F_S = (R_short + R_long + LH + SH) / N_seconds
F_S = F_S[t1, ...]

RF_sum = weighted_sum(R_T+F_S, w_lat, axis=(0, 1), keepdims=False)
energy_residual_final = dTE_sum + RF_sum
print('Residual to conserve energy budge [J/s]: {}'.format(energy_residual_final))